# Mass Transports

Transport diagnostics for flow through major straits.

## Theory

Formally, mass transports are given by
$$T_x = \rho u  $$
$$T_y = \rho v  $$

Mass transports are diagnostics that are calculated online by the model:

|variable|long name|units|dimensions|
|-|
|tx_trans|T-cell i-mass transport|Sv|(time,st_ocean,xu_ocean,yt_ocean)|
|ty_trans|T-cell j-mass transport|Sv|(time,st_ocean,xt_ocean,yu_ocean)|


These variables are saved in `ocean.nc` files.

## Calculation

In [2]:
%matplotlib inline
import cosima_cookbook as cc
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap, shiftgrid
import numpy as np
import netCDF4 as nc
import xarray as xr
import cmocean as cm
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pandas as pd
import copy
import math
from collections import OrderedDict
import IPython.display
from dask.distributed import Client
import cartopy.crs as ccrs
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])


Available exptdata keys:  ['1deg', '025deg', '01deg']


In [3]:
#cc.start_cluster()
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:40101 Dashboard: http://127.0.0.1:40019/status,Cluster Workers: 8 Cores: 8 Memory: 33.67 GB


In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
# use common start and end dates for all runs
tstart = exptdata.clim_tstart
tend = exptdata.clim_tend

lonW = 90.0
lonE = 180.0
latS = -70.0
latN = 20.0

firstyear = pd.to_datetime(tstart).year  # assumes tstart is 1 January!
lastyear = pd.to_datetime(tend).year-1  # assumes tend is 1 January!
yearrange = str(firstyear)+'-'+str(lastyear)
print(yearrange)
print('tstart = ', tstart)
print('tend = ', tend)

output_dir = '/home/561/erd561/figures/Australia/'
fig_name = nb_name[:-6]
fig_dir = output_dir + fig_name

output_ls = os.listdir(output_dir)
if fig_name not in output_ls:
    os.mkdir(fig_dir) 

1993-2017
tstart =  1993-01-01 00:00:00
tend =  2018-01-01 00:00:00


In [6]:
input_path_obs = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

input_path_mod = '/g/data/e14/erd561/access-om2/'

In [7]:
straits_01deg = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('Lombok Strait', [-244.3, -243.8, -8.37, -8.37]),
            ('Ombai Strait',  [-234.9, -234.9, -8.86, -8.27]),
            ('Timor Strait',  [-232.6, -232.6, -13.96, -8.37])
            ])
straits_025deg = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('Lombok Strait', [-244.5, -243.5, -8.22, -8.22]),
            ('Ombai Strait',  [-235.5, -235.5, -9.21, -8.47]),
            ('Timor Strait',  [-233, -233, -13.86, -8.71])
            ])
straits_1deg = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('Lombok Strait', [-245, -242, -8.33, -8.33]),
            ('Ombai Strait',  [-237, -237, -10.33, -8.33]),
            ('Timor Strait',  [-233, -233, -13.9, -9])
            ])


lonW_LC = 110
lonE_LC = 119
latS_LC = -34
latN_LC = -20

latN_157E = -20

lonW_EAC = 147
lonE_EAC = 157
latS_EAC = -47
latN_EAC = -33


LC = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('LC_north Strait', [lonW_LC-360, lonE_LC-360, latN_LC, latN_LC]),
            ('LC_west Strait', [lonW_LC-360, lonW_LC-360, latS_LC, latN_LC]),
            ('LC_south Strait', [lonW_LC-360, lonE_LC-360, latS_LC, latS_LC])
            ])

EAC = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('EAC_north Strait', [lonW_EAC-360, lonE_EAC-360, latN_EAC, latN_EAC]),
            ('EAC_east Strait', [lonE_EAC-360, lonE_EAC-360, latS_EAC, latN_EAC]),
            ('EAC_south Strait', [lonW_EAC-360, lonE_EAC-360, latS_EAC, latS_EAC]),
            ('EAC_west Strait', [lonW_EAC-360, lonW_EAC-360, latS_EAC, -42.2])
            ])

line_157E = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('line_157E Strait', [lonE_EAC-360, lonE_EAC-360, latN_EAC, latN_157E])
            ])


In [8]:
input_path_obs = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path_obs = '/g/data/e14/erd561/NOAA_OI_SST_V2/'
output_path_mod = '/g/data/e14/erd561/access-om2/'

In [17]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_north_100_1 = cc.get_nc_variable(expt, 'ocean.nc', 'ty_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_north_100_1.load()

trans_LC_north_100_1 = trans_LC_north_100_1.groupby('time.year').mean('time', skipna=True)
print(trans_LC_north_100_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_north_100_025 = cc.get_nc_variable(expt, 'ocean.nc', 'ty_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_north_100_025.load()

trans_LC_north_100_025 = trans_LC_north_100_025.groupby('time.year').mean('time', skipna=True)
print(trans_LC_north_100_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_north_100_01_month = cc.get_nc_variable(expt, 'ocean.nc', 'ty_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_north_100_01_month.load()
print(trans_LC_north_100_01_month)

trans_LC_north_100_01 = trans_LC_north_100_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_LC_north_100_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_LC_north_100_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean.nc', 'ty_trans',
                        time_units=time_units).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_north_100_JRA_month.load()



trans_LC_north_100_JRA1 = trans_LC_north_100_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)
trans_LC_north_100_JRA1 = trans_LC_north_100_JRA1.assign_coords(year = trans_LC_north_100_JRA1.year.values + 1944)
print(trans_LC_north_100_JRA1.year.values)



trans_LC_north_100_JRA2 = trans_LC_north_100_JRA_month.isel(time=slice(446,-8))
trans_LC_north_100_JRA2 = trans_LC_north_100_JRA2.assign_coords(time = pd.to_datetime(trans_LC_north_100_JRA2.time.values))
trans_LC_north_100_JRA2 = trans_LC_north_100_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_LC_north_100_JRA2.year.values)
trans_LC_north_100_JRA2 = trans_LC_north_100_JRA2.assign_coords(year = trans_LC_north_100_JRA2.year.values + 44)
print(trans_LC_north_100_JRA2.year.values)



trans_LC_north_100_JRA = xr.concat([trans_LC_north_100_JRA1, trans_LC_north_100_JRA2], dim='year')
print(trans_LC_north_100_JRA)

trans_LC_north_100_1_xr = xr.DataArray(trans_LC_north_100_1, name='trans_LC_north_100_1',
                        coords=[trans_LC_north_100_1.year], 
                        dims=['year'])
print(trans_LC_north_100_1_xr)
trans_LC_north_100_1_xr.to_netcdf(output_path_mod + 'trans_LC_north_100_1.nc')


trans_LC_north_100_025_xr = xr.DataArray(trans_LC_north_100_025, name='trans_LC_north_100_025',
                        coords=[trans_LC_north_100_025.year], 
                        dims=['year'])
print(trans_LC_north_100_025_xr)
trans_LC_north_100_025_xr.to_netcdf(output_path_mod + 'trans_LC_north_100_025.nc')


trans_LC_north_100_01_xr = xr.DataArray(trans_LC_north_100_01, name='trans_LC_north_100_01',
                        coords=[trans_LC_north_100_01.year], 
                        dims=['year'])
print(trans_LC_north_100_01_xr)
trans_LC_north_100_01_xr.to_netcdf(output_path_mod + 'trans_LC_north_100_01.nc')


trans_LC_north_100_JRA_xr = xr.DataArray(trans_LC_north_100_JRA, name='trans_LC_north_100_JRA',
                        coords=[trans_LC_north_100_JRA.year], 
                        dims=['year'])
print(trans_LC_north_100_JRA_xr)
trans_LC_north_100_JRA_xr.to_netcdf(output_path_mod + 'trans_LC_north_100_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans' (year: 300)>
array([-1.082036e+09, -4.425706e+08, -9.876730e+08, ..., -5.402102e+08,
       -1.132200e+09, -1.075149e+09], dtype=float32)
Coordinates:
    yu_ocean  float64 -19.76
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans' (year: 300)>
array([-1.036515e+09, -2.440547e+08, -8.278685e+08, ..., -5.838682e+08,
       -1.227000e+09, -1.210191e+09], dtype=float32)
Coordinates:
    yu_ocean  float64 -20.08
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
01deg_jra

In [19]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_west_100_1 = cc.get_nc_variable(expt, 'ocean.nc', 'tx_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3])).\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('yt_ocean')

trans_LC_west_100_1.load()

trans_LC_west_100_1 = trans_LC_west_100_1.groupby('time.year').mean('time', skipna=True)
print(trans_LC_west_100_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_west_100_025 = cc.get_nc_variable(expt, 'ocean.nc', 'tx_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3])).\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('yt_ocean')

trans_LC_west_100_025.load()

trans_LC_west_100_025 = trans_LC_west_100_025.groupby('time.year').mean('time', skipna=True)
print(trans_LC_west_100_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_west_100_01_month = cc.get_nc_variable(expt, 'ocean.nc', 'tx_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3])).\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('yt_ocean')

trans_LC_west_100_01_month.load()
print(trans_LC_west_100_01_month)

trans_LC_west_100_01 = trans_LC_west_100_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_LC_west_100_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_LC_west_100_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean.nc', 'tx_trans',
                        time_units=time_units).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3])).\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('yt_ocean')

trans_LC_west_100_JRA_month.load()



trans_LC_west_100_JRA1 = trans_LC_west_100_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)-273.15
trans_LC_west_100_JRA1 = trans_LC_west_100_JRA1.assign_coords(year = trans_LC_west_100_JRA1.year.values + 1944)
print(trans_LC_west_100_JRA1.year.values)



trans_LC_west_100_JRA2 = trans_LC_west_100_JRA_month.isel(time=slice(446,-8))
trans_LC_west_100_JRA2 = trans_LC_west_100_JRA2.assign_coords(time = pd.to_datetime(trans_LC_west_100_JRA2.time.values))
trans_LC_west_100_JRA2 = trans_LC_west_100_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_LC_west_100_JRA2.year.values)
trans_LC_west_100_JRA2 = trans_LC_west_100_JRA2.assign_coords(year = trans_LC_west_100_JRA2.year.values + 44)
print(trans_LC_west_100_JRA2.year.values)



trans_LC_west_100_JRA = xr.concat([trans_LC_west_100_JRA1, trans_LC_west_100_JRA2], dim='year')
print(trans_LC_west_100_JRA)

trans_LC_west_100_1_xr = xr.DataArray(trans_LC_west_100_1, name='trans_LC_west_100_1',
                        coords=[trans_LC_west_100_1.year], 
                        dims=['year'])
print(trans_LC_west_100_1_xr)
trans_LC_west_100_1_xr.to_netcdf(output_path_mod + 'trans_LC_west_100_1.nc')


trans_LC_west_100_025_xr = xr.DataArray(trans_LC_west_100_025, name='trans_LC_west_100_025',
                        coords=[trans_LC_west_100_025.year], 
                        dims=['year'])
print(trans_LC_west_100_025_xr)
trans_LC_west_100_025_xr.to_netcdf(output_path_mod + 'trans_LC_west_100_025.nc')


trans_LC_west_100_01_xr = xr.DataArray(trans_LC_west_100_01, name='trans_LC_west_100_01',
                        coords=[trans_LC_west_100_01.year], 
                        dims=['year'])
print(trans_LC_west_100_01_xr)
trans_LC_west_100_01_xr.to_netcdf(output_path_mod + 'trans_LC_west_100_01.nc')


trans_LC_west_100_JRA_xr = xr.DataArray(trans_LC_west_100_JRA, name='trans_LC_west_100_JRA',
                        coords=[trans_LC_west_100_JRA.year], 
                        dims=['year'])
print(trans_LC_west_100_JRA_xr)
trans_LC_west_100_JRA_xr.to_netcdf(output_path_mod + 'trans_LC_west_100_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans' (year: 300)>
array([2.389163e+09, 2.648528e+09, 3.040654e+09, ..., 2.354636e+09,
       2.079080e+09, 2.759929e+09], dtype=float32)
Coordinates:
    xu_ocean  float64 -250.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans' (year: 300)>
array([2.241203e+09, 3.112901e+09, 2.731629e+09, ..., 2.169475e+09,
       1.582159e+09, 2.239390e+09], dtype=float32)
Coordinates:
    xu_ocean  float64 -250.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
01deg_jra55v13_iaf
No

In [20]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_south_100_1 = cc.get_nc_variable(expt, 'ocean.nc', 'ty_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_south_100_1.load()

trans_LC_south_100_1 = trans_LC_south_100_1.groupby('time.year').mean('time', skipna=True)
print(trans_LC_south_100_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_south_100_025 = cc.get_nc_variable(expt, 'ocean.nc', 'ty_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_south_100_025.load()

trans_LC_south_100_025 = trans_LC_south_100_025.groupby('time.year').mean('time', skipna=True)
print(trans_LC_south_100_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_LC_south_100_01_month = cc.get_nc_variable(expt, 'ocean.nc', 'ty_trans',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_south_100_01_month.load()
print(trans_LC_south_100_01_month)

trans_LC_south_100_01 = trans_LC_south_100_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_LC_south_100_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_LC_south_100_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean.nc', 'ty_trans',
                        time_units=time_units).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest').\
                        sel(st_ocean=slice(0, 100)).\
                        sum('st_ocean').\
                        sum('xt_ocean')

trans_LC_south_100_JRA_month.load()



trans_LC_south_100_JRA1 = trans_LC_south_100_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)
trans_LC_south_100_JRA1 = trans_LC_south_100_JRA1.assign_coords(year = trans_LC_south_100_JRA1.year.values + 1944)
print(trans_LC_south_100_JRA1.year.values)



trans_LC_south_100_JRA2 = trans_LC_south_100_JRA_month.isel(time=slice(446,-8))
trans_LC_south_100_JRA2 = trans_LC_south_100_JRA2.assign_coords(time = pd.to_datetime(trans_LC_south_100_JRA2.time.values))
trans_LC_south_100_JRA2 = trans_LC_south_100_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_LC_south_100_JRA2.year.values)
trans_LC_south_100_JRA2 = trans_LC_south_100_JRA2.assign_coords(year = trans_LC_south_100_JRA2.year.values + 44)
print(trans_LC_south_100_JRA2.year.values)



trans_LC_south_100_JRA = xr.concat([trans_LC_south_100_JRA1, trans_LC_south_100_JRA2], dim='year')
print(trans_LC_south_100_JRA)

trans_LC_south_100_1_xr = xr.DataArray(trans_LC_south_100_1, name='trans_LC_south_100_1',
                        coords=[trans_LC_south_100_1.year], 
                        dims=['year'])
print(trans_LC_south_100_1_xr)
trans_LC_south_100_1_xr.to_netcdf(output_path_mod + 'trans_LC_south_100_1.nc')


trans_LC_south_100_025_xr = xr.DataArray(trans_LC_south_100_025, name='trans_LC_south_100_025',
                        coords=[trans_LC_south_100_025.year], 
                        dims=['year'])
print(trans_LC_south_100_025_xr)
trans_LC_south_100_025_xr.to_netcdf(output_path_mod + 'trans_LC_south_100_025.nc')


trans_LC_south_100_01_xr = xr.DataArray(trans_LC_south_100_01, name='trans_LC_south_100_01',
                        coords=[trans_LC_south_100_01.year], 
                        dims=['year'])
print(trans_LC_south_100_01_xr)
trans_LC_south_100_01_xr.to_netcdf(output_path_mod + 'trans_LC_south_100_01.nc')


trans_LC_south_100_JRA_xr = xr.DataArray(trans_LC_south_100_JRA, name='trans_LC_south_100_JRA',
                        coords=[trans_LC_south_100_JRA.year], 
                        dims=['year'])
print(trans_LC_south_100_JRA_xr)
trans_LC_south_100_JRA_xr.to_netcdf(output_path_mod + 'trans_LC_south_100_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans' (year: 300)>
array([-9.732004e+08, -7.335107e+08, -1.040567e+09, ..., -8.236712e+08,
       -7.999498e+08, -8.910038e+08], dtype=float32)
Coordinates:
    yu_ocean  float64 -33.99
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans' (year: 300)>
array([-1.089874e+09, -1.548772e+09, -1.472849e+09, ..., -1.107436e+09,
       -7.498324e+08, -1.286722e+09], dtype=float32)
Coordinates:
    yu_ocean  float64 -34.05
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
01deg_jra